In [185]:
# Instal EOD historical data API & yield curve contruction module
!pip install eod
!pip install nelson_siegel_svensson

In [186]:
from scipy.integrate import quad
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.integrate import quad
from scipy.optimize import minimize
from datetime import datetime as dt

from eod import EodHistoricalData
from nelson_siegel_svensson import NelsonSiegelSvenssonCurve
from nelson_siegel_svensson.calibrate import calibrate_nss_ols


In [187]:
def heston_charfunc(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):

    # constants
    a = kappa*theta
    b = kappa+lambd

    # common terms w.r.t phi
    rspi = rho*sigma*phi*1j

    # d parameter given phi and b
    d = np.sqrt( (rho*sigma*phi*1j - b)**2 + (phi*1j+phi**2)*sigma**2 )

    # g parameter given phi, b and d
    g = (b-rspi+d)/(b-rspi-d)
    # characteristic function by components
    exp1 = np.exp(r*phi*1j*tau)
    term2 = S0**(phi*1j) * ( (1-g*np.exp(d*tau))/(1-g) )**(-2*a/sigma**2)
    exp2 = np.exp(a*tau*(b-rspi+d)/sigma**2 + v0*(b-rspi+d)*( (1-np.exp(d*tau))/(1-g*np.exp(d*tau)) )/sigma**2)

    return exp1*term2*exp2

def integrand(phi, S0, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)
    numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K*heston_charfunc(phi,*args)
    denominator = 1j*phi*K**(1j*phi)
    return numerator/denominator

def heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

    P, umax, N = 0, 100, 10000
    dphi=umax/N #dphi is width

    for i in range(1,N):
        # rectangular integration
        phi = dphi * (2*i + 1)/2
        numerator = np.exp(r*tau)*heston_charfunc(phi-1j,*args) - K * heston_charfunc(phi,*args)
        denominator = 1j*phi*K**(1j*phi)

        P += dphi * numerator/denominator

    return np.real((S0 - K*np.exp(-r*tau))/2 + P/np.pi)

def heston_price(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r):
    args = (S0, v0, kappa, theta, sigma, rho, lambd, tau, r)

    real_integral, err = np.real( quad(integrand, 0, 100, args=args) )

    return (S0 - K*np.exp(-r*tau))/2 + real_integral/np.pi


In [188]:
# Parameters to test model

S0 = 248. # initial asset price
K = 248. # strike
v0 = 0.1 # initial variance
r = 0.03 # risk free rate
kappa = 1.5768 # rate of mean reversion of variance process
theta = 0.0398 # long-term mean variance
sigma = 0.3 # volatility of volatility
lambd = 0.575 # risk premium of variance
rho = -0.5711 # correlation between variance and stock process
tau = 1. # time to maturity



In [189]:
heston_price( S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r )

/usr/local/lib/python3.11/dist-packages/scipy/integrate/_quadpack_py.py:611: ComplexWarning:

Casting complex values to real discards the imaginary part



28.620097312001285

In [190]:
#heston_price( S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r )
yield_maturities = np.array([1/12, 2/12, 3/12, 6/12, 1, 2, 3, 5, 7, 10, 20, 30])
yeilds = np.array([0.15,0.27,0.50,0.93,1.52,2.13,2.32,2.34,2.37,2.32,2.65,2.52]).astype(float)/100

In [191]:
#NSS model calibrate
curve_fit, status = calibrate_nss_ols(yield_maturities,yeilds)

curve_fit


NelsonSiegelSvenssonCurve(beta0=0.028391531922961333, beta1=-0.029279498864207682, beta2=0.025428219264836193, beta3=-0.01417407230854954, tau1=0.9922983277200589, tau2=4.78140939601007)

###new fetch function

In [192]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime as dt, timedelta

def fetch_options_data(ticker_symbol):
    ticker = yf.Ticker(ticker_symbol)
    expiration_dates = ticker.options

    exp_dates_dt = [dt.strptime(date, "%Y-%m-%d") for date in expiration_dates]
    today = dt.today()

    # Define target maturities
    target_maturities = [30, 90, 180, 365]  # Days (1 month, 3 months, etc.)

    # Select expiration dates closest to the target maturities
    selected_exp_dates = []
    for target in target_maturities:
        closest_date = min(exp_dates_dt, key=lambda x: abs((x - today).days - target))
        if closest_date not in selected_exp_dates:
            selected_exp_dates.append(closest_date)


    selected_exp_dates = [date.strftime("%Y-%m-%d") for date in selected_exp_dates]

    all_calls_data = []
    for exp_date in selected_exp_dates:
        option_chain = ticker.option_chain(exp_date)
        calls_df = option_chain.calls.copy()
        calls_df["expiration_date"] = exp_date
        all_calls_data.append(calls_df)

    options_data = pd.concat(all_calls_data, ignore_index=True)

    return options_data, selected_exp_dates


In [193]:
call_df, expiration_date = fetch_options_data("AAPL")
call_df

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration_date
0,AAPL250404C00100000,2025-02-20 14:30:04+00:00,100.0,146.58,140.00,143.85,0.00,0.000000,NaN,0,1.426761,True,REGULAR,USD,2025-04-04
1,AAPL250404C00140000,2025-02-24 19:37:23+00:00,140.0,108.87,100.25,104.00,0.00,0.000000,1.0,2,0.978028,True,REGULAR,USD,2025-04-04
2,AAPL250404C00180000,2025-02-27 19:32:53+00:00,180.0,61.36,60.65,64.35,0.00,0.000000,1.0,2,0.630131,True,REGULAR,USD,2025-04-04
3,AAPL250404C00185000,2025-02-14 18:08:42+00:00,185.0,60.83,55.75,58.45,0.00,0.000000,NaN,2,0.528569,True,REGULAR,USD,2025-04-04
4,AAPL250404C00190000,2025-02-20 16:26:53+00:00,190.0,57.40,50.80,54.50,0.00,0.000000,NaN,2,0.551274,True,REGULAR,USD,2025-04-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,AAPL260320C00320000,2025-02-28 20:44:10+00:00,320.0,4.71,4.25,5.05,0.16,3.516480,101.0,240,0.252815,False,REGULAR,USD,2026-03-20
158,AAPL260320C00330000,2025-02-26 14:36:18+00:00,330.0,3.80,3.40,3.70,0.00,0.000000,2.0,390,0.246590,False,REGULAR,USD,2026-03-20
159,AAPL260320C00340000,2025-02-28 18:37:47+00:00,340.0,2.41,2.41,3.10,-0.24,-9.056604,6.0,61,0.250343,False,REGULAR,USD,2026-03-20
160,AAPL260320C00350000,2025-02-27 20:21:01+00:00,350.0,2.00,1.81,2.14,0.00,0.000000,1.0,419,0.242500,False,REGULAR,USD,2026-03-20


In [194]:
expiration_date

['2025-04-04', '2025-05-16', '2025-08-15', '2026-03-20']

In [195]:
call_df.columns

Index(['contractSymbol', 'lastTradeDate', 'strike', 'lastPrice', 'bid', 'ask',
       'change', 'percentChange', 'volume', 'openInterest',
       'impliedVolatility', 'inTheMoney', 'contractSize', 'currency',
       'expiration_date'],
      dtype='object')

In [196]:
S0 = yf.Ticker("AAPL").history(period="1d")["Close"].iloc[-1]
# Market prices dictionary
market_prices = {}

# Iterate through expiration dates and store data in the dictionary
for date in expiration_date:
    # Filter call_df for the current expiration date
    current_call_df = call_df[call_df['expiration_date'] == date]

    # Extract relevant data and store in market_prices
    market_prices[date] = {  # Use date as the key
        'strike': current_call_df['strike'].tolist(),
        'price': ((current_call_df['bid'] + current_call_df['ask']) / 2).tolist()
    }

In [197]:
# Finding common strike prices
all_strikes = [v['strike'] for v in market_prices.values()]
common_strikes = set.intersection(*map(set, all_strikes))
print('Number of common strikes:', len(common_strikes))
common_strikes = sorted(common_strikes)


Number of common strikes: 17


In [198]:
# Extract necessary values
prices = []
maturities = []
for date, v in market_prices.items():
    maturities.append((dt.strptime(date, '%Y-%m-%d') - dt.today()).days / 365.25)
    price = [v['price'][i] for i, x in enumerate(v['strike']) if x in common_strikes]
    prices.append(price)

# Convert to numpy array
price_arr = np.array(prices, dtype=object)
np.shape(price_arr)

(4, 17)

In [199]:
S0

241.83999633789062

In [200]:
# Create volatility surface DataFrame
volSurface = pd.DataFrame(price_arr, index=maturities, columns=common_strikes)

# Convert index and columns to numeric if they are not already
volSurface.index = pd.to_numeric(volSurface.index, errors='coerce')
volSurface.columns = pd.to_numeric(volSurface.columns, errors='coerce')

#new
# Define valid range for maturities (7 days to 1 year)
volSurface = volSurface.loc[(volSurface.index > 7/365) & (volSurface.index <= 1)]

# Define valid range for strike prices (±50% of spot price)
spot_price = price_arr[0, 0]  # Assuming the first row, first column is the stock price
volSurface = volSurface.loc[:, (volSurface.columns > 0.5 * spot_price) & (volSurface.columns < 1.5 * spot_price)]

# Convert to long format
if not volSurface.empty:
    volSurfaceLong = volSurface.melt(ignore_index=False).reset_index()
    volSurfaceLong.columns = ['maturity', 'strike', 'price']
else:
    print("Warning: Filtered volSurface is empty!")

# Print output to check
print("VolSurfaceLong:\n", volSurfaceLong.head())

VolSurfaceLong:
    maturity  strike    price
0  0.087611   140.0  102.125
1  0.202601   140.0   103.15
2  0.451745   140.0   104.95


In [201]:
volSurfaceLong = pd.DataFrame(price_arr, index=maturities, columns=common_strikes)
volSurfaceLong


,140.0,180.0,185.0,190.0,195.0,200.0,210.0,220.0,230.0,240.0,250.0,260.0,270.0,280.0,290.0,300.0,310.0
0.087611,102.125,62.5,57.1,52.65,47.725,42.275,33.375,23.95,15.575,8.675,3.95,1.235,0.395,0.12,0.35,0.04,0.305
0.202601,103.15,63.95,59.05,54.3,50.1,44.9,36.075,27.925,19.9,13.6,8.0,4.325,2.3,1.04,0.505,0.265,0.175
0.451745,104.95,67.55,62.325,57.8,53.4,49.375,41.3,33.425,26.275,20.425,14.35,10.375,7.075,4.45,2.93,1.705,1.02
1.045859,108.65,74.0,69.775,66.2,62.2,57.85,50.85,43.775,37.775,30.925,25.525,20.8,16.725,13.3,11.125,8.075,6.275


In [202]:
# Ensure volSurface is not empty before melting
if not volSurface.empty:
    volSurfaceLong = volSurface.melt(ignore_index=False).reset_index()
    volSurfaceLong.columns = ['maturity', 'strike', 'price']
    # Calculate risk-free rate only if volSurfaceLong has data
    #volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit)
else:
    print("Warning: Filtered volSurface is empty!")
    #volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit)

# Print output to check
print("VolSurfaceLong:\n", volSurfaceLong.head())

VolSurfaceLong:
    maturity  strike    price
0  0.087611   140.0  102.125
1  0.202601   140.0   103.15
2  0.451745   140.0   104.95


In [203]:
volSurface

,140.0
0.087611,102.125
0.202601,103.15
0.451745,104.95


In [204]:

# Calculate risk-free rate
volSurfaceLong['rate'] = volSurfaceLong['maturity'].apply(curve_fit)

In [205]:
# Define Heston model parameters
S0 = call_df['lastPrice'].iloc[0]  # Underlying stock price
r = volSurfaceLong['rate'].to_numpy('float')
K = volSurfaceLong['strike'].to_numpy('float')
tau = volSurfaceLong['maturity'].to_numpy('float')
P = volSurfaceLong['price'].to_numpy('float')

params = {
    "v0": {"x0": 0.1, "lbub": [1e-3, 0.1]},
    "kappa": {"x0": 3, "lbub": [1e-3, 5]},
    "theta": {"x0": 0.05, "lbub": [1e-3, 0.1]},
    "sigma": {"x0": 0.3, "lbub": [1e-2, 1]},
    "rho": {"x0": -0.8, "lbub": [-1, 0]},
    "lambd": {"x0": 0.03, "lbub": [-1, 1]},
}

x0 = [param["x0"] for param in params.values()]
bnds = [param["lbub"] for param in params.values()]

# Objective function for optimization
def SqErr(x):
    v0, kappa, theta, sigma, rho, lambd = x
    err = np.sum((P - heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r))**2 / len(P))
    return err

In [206]:
# Optimize parameters
result = minimize(SqErr, x0, tol=1e-3, method='SLSQP', options={'maxiter': 1e4}, bounds=bnds)
result

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:437: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds

/usr/local/lib/python3.11/dist-packages/scipy/optimize/_slsqp_py.py:441: RuntimeWarning:

Values in x were outside bounds during a minimize step, clipping to bounds



 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 8190.52174683166
       x: [ 1.000e-01  1.000e-03  1.000e-01  1.000e+00 -1.000e+00
           -1.000e+00]
     nit: 2
     jac: [-8.718e+03 -2.298e+01 -1.262e+00  3.116e+02  2.285e+03
            1.033e+02]
    nfev: 14
    njev: 2

In [207]:
v0, kappa, theta, sigma, rho, lambd = [param for param in result.x]
v0, kappa, theta, sigma, rho, lambd

(0.1, 0.0009999999999998899, 0.1, 1.0, -1.0, -1.0)

###calculate heston prices

In [208]:
heston_prices = heston_price_rec(S0, K, v0, kappa, theta, sigma, rho, lambd, tau, r)

In [209]:
volSurfaceLong['heston_price'] = heston_prices
volSurfaceLong

,maturity,strike,price,rate,heston_price
0,0.087611,140.0,102.125,0.001298,9.708464
1,0.202601,140.0,103.15,0.003884,12.547736
2,0.451745,140.0,104.95,0.008540,16.508048
